# Data refinement

With this Jupyter Notebook i can do diferent operations in the original images i have to resize, move or translate for my Neural Network.

I have a lot of videos and i m going to take frames from them. I will refine that frames to have the images for our Keras Neural Network

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import cv2
import os,shutil
import random
import time

def printAndRemove(msg):
    print(msg, end="\r")
    time.sleep(0)

# Extract images from videos

With this code i will take the images from my videos:

In [11]:
#size = 1920, 1080
#size = 960, 540
#size = 480, 270
#size = 240, 135
size = 128, 72
#size = 96, 54

path = ""
saveIn = "data"

if os.path.isdir(path + saveIn):
    shutil.rmtree(path + saveIn)
os.makedirs(path + saveIn)
os.makedirs(path + saveIn + "/train")
os.makedirs(path + saveIn + "/validation")

i=0

imgNumber = 0
trainingPerNode = []
numberImagesInThisNode = 0
for video in glob.glob(path + "Nodes/*.mp4"):
    i=i+1
    newDir = video.split('.')[0].split('_')[0].split('\\')
    newDir = newDir[len(newDir)-1]
    if not os.path.exists(path + saveIn + "/train/" + newDir):
        numberImagesInThisNode = 0
        os.makedirs(path + saveIn + "/train/" + newDir)
    if not os.path.exists(path + saveIn + "/validation/" + newDir):
        os.makedirs(path + saveIn + "/validation/" + newDir)
    
    cap = cv2.VideoCapture(video)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    currentframe = 0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            printAndRemove("Working on image nº" + str(imgNumber))
            
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            if(i != 15):
                frame = frame.rotate(180)
            frame.thumbnail(size, Image.ANTIALIAS)
            frame.save(path + saveIn + "/train/" + newDir + "/img" + str(imgNumber) + ".jpg", "JPEG")
            
            numberImagesInThisNode += 1
            imgNumber += 1
        else:
            break
    trainingPerNode.append(numberImagesInThisNode)
cap.release()
cv2.destroyAllWindows()

validationPerNode = []
node=0
for directory in glob.glob(path + saveIn + "/train/*"):
    directory = directory.split('\\')[1]
    validationImages = 0
    while validationImages < trainingPerNode[node]/10:
        for img in glob.glob(path + saveIn + "/train/" + directory + "/*.jpg"):
            if(validationImages < trainingPerNode[node]/10):
                r = random.randint(0, 100)
                if(r<15):
                    shutil.move(img, path + saveIn + "/validation/" + directory)
                    validationImages += 1
    node += 1
    validationPerNode.append(validationImages)


print("")
print("Done!")

Working on image nº33227
Done!


# Count images

With this code i will count all the data i have

In [12]:
path = ""
saveIn = "data"

def countImages(directory):
    count=0
    for img in glob.glob(directory + "/*.jpg"):
        count = count + 1
    return count

training = []
print("Training\n")
for directory in glob.glob(path + saveIn + "/train/*"):
    numImages = countImages(directory)
    nodeName = directory.split('\\')[1]
    print(nodeName + "")
    print("\tTotal images in "+ str(nodeName) +": " +  str(numImages))
    training.append(numImages)

print("\nValidation\n")
for directory in glob.glob(path + saveIn + "/validation/*"):
    numImages = countImages(directory)
    nodeName = directory.split('\\')[1]
    print(nodeName + "")
    print("\tTotal images in "+ str(nodeName) +": " +  str(numImages))

Training

Nodo1
	Total images in Nodo1: 5169
Nodo2
	Total images in Nodo2: 2189
Nodo3
	Total images in Nodo3: 4058
Nodo4
	Total images in Nodo4: 2781
Nodo5
	Total images in Nodo5: 11156
Nodo6
	Total images in Nodo6: 1674
Nodo7
	Total images in Nodo7: 3731

Validation

Nodo1
	Total images in Nodo1: 121
Nodo2
	Total images in Nodo2: 228
Nodo3
	Total images in Nodo3: 316
Nodo4
	Total images in Nodo4: 340
Nodo5
	Total images in Nodo5: 426
Nodo6
	Total images in Nodo6: 510
Nodo7
	Total images in Nodo7: 529


# Disperse images

With this code i will disperse all the images i have. With this i expect to dont have our images in the training in a sequential order and improve the training

In [13]:
import random

def disperse(directory):
    # We create the temporal file
    if os.path.isdir(directory + "temp"):
        shutil.rmtree(directory + "temp")
    os.makedirs(directory + "temp")
    # We count the images
    numberOfImagesInDirectory = countImages(directory)
    # We change the name of all the images in the file
    for image in glob.glob(directory + "/*.jpg"):
        nameIsUsed = True
        newName = ""
        # No collision with other images names
        while(nameIsUsed):
            newName = "img" + str(numberOfImagesInDirectory + random.randint(0, 100000))
            if(os.path.isfile(directory + "temp/" + newName + ".jpg")):
                nameIsUsed = True
            else:
                nameIsUsed = False
        # We rename the file
        os.rename(image, directory + "temp/" + newName + ".jpg")
    for image in glob.glob(directory + "temp/*.jpg"):
        temporal = image.split("temp")
        shutil.move(image,temporal[0]+temporal[1])
    shutil.rmtree(directory + "temp")

for directory in glob.glob(path + saveIn + "/train/*"):
    disperse(directory)

training = []
print("Training\n")
for directory in glob.glob(path + saveIn + "/train/*"):
    numImages = countImages(directory)
    nodeName = directory.split('\\')[1]
    print(nodeName + "")
    print("\tTotal images in "+ str(nodeName) +": " +  str(numImages))
    training.append(numImages)

Training

Nodo1
	Total images in Nodo1: 5169
Nodo2
	Total images in Nodo2: 2189
Nodo3
	Total images in Nodo3: 4058
Nodo4
	Total images in Nodo4: 2781
Nodo5
	Total images in Nodo5: 11156
Nodo6
	Total images in Nodo6: 1674
Nodo7
	Total images in Nodo7: 3731
